In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.ticker as mtick
import scipy.signal as signal
import imageio as io
from tqdm import tqdm
import pickle, re
import cv2
font = cv2.FONT_HERSHEY_SIMPLEX
rgb = (255, 255, 255)

import sys
sys.path.insert(1, '../numphly')
import numphly as nly
import mcam_tools
import tracker_decoder as tkr_dec
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


def check_characters(input_string, target_characters):
    return set(target_characters).issubset(set(input_string))


In [2]:
exp_target = 'R72'
gtype_name = ['R72A10xTRPA1','R72A10xW1118','W1118xTRPA1']
sex_name = ['female','male']
h5_dir = '../../FlyBehaviorSet/DLC_tracking/'+exp_target
output_fileloc = '../../FlyBehaviorSet/'
OG_recording_folder      = 'original_recording_for_tracking/'+exp_target
converted_folder         = 'Converted/R72_videos/'
cropped_recording_folder = 'cropped_recording_for_tracking'  
dir_MCAM = '../DatastreamLine_MCAM_Tracking/'
shared_data_folder = '../Shared_data/'

scorer_floor  = 'DLC_resnet152_floor_topview_12label_1uniqueJan4shuffle1_100000'
# scorer_wall   = 'DLC_resnet152_wall_sideview_7labelsJan1shuffle1_100000'
scorer_wall   = 'DLC_resnet152_wall_sideview_7labelsJan1shuffle1_90000'
# scorer_cross  = 'DLC_resnet152_wall_cross_section_view_8labelFeb19shuffle1_100000'
scorer_cross  = 'DLC_resnet152_wall_cross_section_view_8labelFeb19shuffle1_30000'


pcutoff = 0
frame_n = 126000
frame_rate = 60
frame_n_e = frame_rate*60*30

color_BP = [(0,0,255),(0,255,0),(255,0,0)]
arena_parameters = {
    'well_radius': 8.4,  # mm
    'food_cup_radius': 3.5,  # mm
}

experiments_index  = np.load('experiments_index.npy')
fileloc_set        = np.load('experiments_directory.npy')
num_file = len(fileloc_set)
videofile_set      = np.load('experiments_well_setup.npy')
num_well = len(videofile_set)
experiments_mscore = np.load('experiments_mscore.npy',allow_pickle='TRUE').item()
crop_h = np.load('crop_h.npy')
print(len(fileloc_set))

15


In [3]:
fileloc = fileloc_set[0]
experiment_index = experiments_index[0]
well = 'v_'+experiment_index+'_'+videofile_set[0]
numBP_floor = 12
numBP_wall  = 7
numBP_cross = 8
numUniq = 1

hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
Dataframe = pd.read_hdf(os.path.join(hfile_floor))
bodyparts = Dataframe.columns.get_level_values(2)[0:3*numBP_floor];
bodyparts_floor = (bodyparts[0::3])

uniqueID = Dataframe.columns.get_level_values(1)[0:3*numBP_floor*3];
uniqueID = (uniqueID[0::3*numBP_floor])[2]
paint = Dataframe.columns.get_level_values(2)[0:3*numBP_floor*3];
paint = paint[3*numBP_floor*2]

hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
Dataframe = pd.read_hdf(os.path.join(hfile_wall))
bodyparts = Dataframe.columns.get_level_values(2)[0:3*numBP_wall];
bodyparts_wall = (bodyparts[0::3])

hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
Dataframe = pd.read_hdf(os.path.join(hfile_cross))
bodyparts = Dataframe.columns.get_level_values(2)[0:3*numBP_cross];
bodyparts_cross = (bodyparts[0::3])


animalID = Dataframe.columns.get_level_values(1)[0:3*numBP_cross*3];
animalID = (animalID[0::3*numBP_cross])#[0:2]
n_ID = len(animalID)

# numBP_floor = 5
# numBP_wall  = 4
# numBP_cross = 4
# bodyparts_floor = bodyparts_floor[0:numBP_floor]
# bodyparts_wall  = bodyparts_wall[0:numBP_wall]
# bodyparts_cross = bodyparts_cross[[0,3,4,7]]

# numBP_floor = 5
# numBP_wall  = 4
# numBP_cross = 4
# bodyparts_floor = bodyparts_floor[[0,1,2,3,4]]
# bodyparts_wall  = bodyparts_wall[[0,1,2,3]]
# bodyparts_cross = bodyparts_cross[[0,3,4,7]]

# numBP_floor = 3
# numBP_wall  = 3
# numBP_cross = 4
# bodyparts_floor = bodyparts_floor[[0,3,4]]
# bodyparts_wall  = bodyparts_wall[[1,2,3]]
# bodyparts_cross = bodyparts_cross[[0,3,4,7]]

numBP_floor = 3
numBP_wall  = 3
numBP_cross = 4
bodyparts_floor = bodyparts_floor[[0,3,4]]
bodyparts_wall  = bodyparts_wall[[1,2,3]]
bodyparts_cross = bodyparts_cross[[0,3,4,7]]


# numBP_floor = 2
# numBP_wall  = 2
# numBP_cross = 4
# bodyparts_floor = bodyparts_floor[[0,4]]
# bodyparts_wall  = bodyparts_wall[[1,3]]
# bodyparts_cross = bodyparts_cross[[0,3,4,7]]


# print(bodyparts_floor1)
print(uniqueID,paint)
print(bodyparts_floor, len(bodyparts_floor))
print(bodyparts_wall, len(bodyparts_wall))
print(bodyparts_cross, len(bodyparts_cross))
# print(animalID)

num_exp = 0
for i in range(num_file):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    for j in range(num_well):
        sex = sexes[j]

        if sex != -1:
            num_exp = num_exp+1
            
            
total_events = np.zeros((3,num_exp)).astype(int)
genders_status   = -np.ones((num_exp)).astype(int)
genotypes_status = -np.ones((num_exp)).astype(int)
radius = -np.ones((num_exp))

ii = 0
for i in range(num_file):
    experiment  = experiments_mscore[fileloc_set[i]]
    sexes       = experiment['sex']
    genotypes   = experiment['genotype']
    events_type = experiment['events type']
    events      = experiment['events']
    circle      = experiment['arena circle']
    start_t     = np.max(experiment['starting_time'])
    well_tracker_select = np.logical_and(np.arange(frame_n)>=start_t,np.arange(frame_n)<start_t+frame_n_e)
    experiment_index = experiments_index[i]
    for j in range(num_well):

        sex = sexes[j]
        genotype = genotypes[j]

        if sex != -1:
#         if sex == 0 and genotype == 2:
            event_ts   = np.array(events[j]).astype(int)
            event_type = np.array(events_type[j]).astype(int)
            well_arena = circle[j]

            n_event = len(event_ts)
            n_event_HB  = sum(event_type==1)
            n_event_UWE = sum(event_type==2)
            n_event_L   = sum(event_type==3)
            
            n_event_HB_s  = sum(well_tracker_select[event_ts[event_type==1]])
            n_event_UWE_s = sum(well_tracker_select[event_ts[event_type==2]])
            n_event_L_s   = sum(well_tracker_select[event_ts[event_type==3]])
            
            total_events[0,ii] = n_event_HB_s
            total_events[1,ii] = n_event_UWE_s
            total_events[2,ii] = n_event_L_s

            if n_event_HB != n_event_HB_s:
                print(experiment_index+'_'+videofile_set[j],'HB mis:',n_event_HB, n_event_HB_s)
            if n_event_UWE != n_event_UWE_s:
                print(experiment_index+'_'+videofile_set[j],'UWE mis:',n_event_UWE, n_event_UWE_s)
            if n_event_L != n_event_L_s:
                print(experiment_index+'_'+videofile_set[j],'L mis:',n_event_L, n_event_L_s)

            
            genders_status[ii] = sex
            genotypes_status[ii] = genotype
            radius[ii] = (np.mean(well_arena, axis = 0))[2]
#             print(experiment_index+'_'+videofile_set[j], total_events[0,ii], total_events[1,ii], total_events[2,ii])
            ii += 1
print(ii)



single paint
Index(['head', 'center', 'tail'], dtype='object', name='bodyparts') 3
Index(['head', 'center', 'tail'], dtype='object', name='bodyparts') 3
Index(['top', 'left', 'right', 'down'], dtype='object', name='bodyparts') 4
05_08_24_exp1_D1 UWE mis: 15 14
05_08_24_exp2_C1 HB mis: 58 54
05_08_24_exp2_D1 HB mis: 40 38
05_08_24_exp2_A2 L mis: 65 57
05_08_24_exp2_D2 HB mis: 65 56
05_09_24_exp1_A1 UWE mis: 25 17
05_09_24_exp1_C1 HB mis: 40 39
05_09_24_exp1_A2 UWE mis: 18 17
05_09_24_exp1_B2 HB mis: 26 25
05_09_24_exp2_B1 HB mis: 8 6
05_09_24_exp2_D1 L mis: 32 27
05_09_24_exp2_A2 HB mis: 39 37
05_13_24_exp1_A1 UWE mis: 91 84
05_13_24_exp1_D1 UWE mis: 50 43
05_13_24_exp1_D1 L mis: 5 2
05_13_24_exp1_D2 HB mis: 26 25
05_13_24_exp2_B1 HB mis: 49 47
05_13_24_exp2_C1 UWE mis: 14 4
05_13_24_exp2_C1 L mis: 61 42
05_13_24_exp2_C2 UWE mis: 105 101
05_13_24_exp3_A1 UWE mis: 77 73
05_13_24_exp3_B1 HB mis: 76 72
05_13_24_exp3_D1 UWE mis: 51 42
05_15_24_exp1_B2 UWE mis: 2 1
05_17_24_exp1_A1 HB mis: 9

# locomotion datas

In [4]:
np.set_printoptions(precision=2)

# window_size = np.arange(30)+1
window_size = [1,6,60]
# window_size = [1,2,60]
# window_size = [30,60]
# window_size = [1,6]
l_ws = len(window_size)

filter_status = '_filtered'
# filter_status = ''

thr_presence = 0.1
size_npz = shared_data_folder+exp_target+'_bodysize_event.npz'
loaded_size = np.load(size_npz)

d_f1f2_m_tracked, t_f1f2_m_tracked, ad_d_f1f2_m_tracked, t_ad_d_f1f2_m_tracked, t_f1f2_c_tracked, d_f1f2_tracked, t_f1f2_tracked, d_tracked, t_speed_tracked,norm_f1_f2_face_m, norm_f1_f2_chase_m, t_face_tracked, t_chase_tracked,norm_f1_f2_face_m_wocr,norm_f1_f2_chase_m_wocr, t_face_wocr_tracked, t_chase_wocr_tracked = (mcam_tools.all_tracking_data(
    h5_dir, loaded_size, filter_status, scorer_floor, scorer_wall, scorer_cross,experiments_mscore, experiments_index, fileloc_set, videofile_set, 
    animalID, bodyparts_floor, num_file,num_well, num_exp, window_size, crop_h, frame_n, frame_n_e, thr_presence))


# d_f1f2_m_tracked, t_f1f2_m_tracked, ad_d_f1f2_m_tracked, t_ad_d_f1f2_m_tracked, t_f1f2_c_tracked, d_f1f2_tracked, t_f1f2_tracked, d_tracked, t_speed_tracked,norm_f1_f2_face_m, norm_f1_f2_chase_m, t_face_tracked, t_chase_tracked,norm_f1_f2_face_m_wocr,norm_f1_f2_chase_m_wocr, t_face_wocr_tracked, t_chase_wocr_tracked = (all_tracking_data(
#     h5_dir, loaded_size, filter_status, scorer_floor, scorer_wall, scorer_cross,experiments_mscore, experiments_index, fileloc_set, videofile_set, 
#     animalID, bodyparts_floor, num_file,num_well, num_exp, window_size, crop_h, frame_n, frame_n_e, thr_presence))




v_05_08_24_exp1_A1 1 0.879 0.879 0.879
v_05_08_24_exp1_A1 6 0.914 0.914 0.914
v_05_08_24_exp1_A1 60 0.951 0.951 0.951
v_05_08_24_exp1_B1 1 0.920 0.920 0.920
v_05_08_24_exp1_B1 6 0.950 0.950 0.950
v_05_08_24_exp1_B1 60 0.974 0.974 0.974
v_05_08_24_exp1_D1 1 0.731 0.731 0.731
v_05_08_24_exp1_D1 6 0.836 0.836 0.836
v_05_08_24_exp1_D1 60 0.943 0.943 0.943
v_05_08_24_exp1_A2 1 0.950 0.950 0.950
v_05_08_24_exp1_A2 6 0.976 0.976 0.976
v_05_08_24_exp1_A2 60 0.994 0.994 0.994
v_05_08_24_exp1_B2 1 0.865 0.865 0.865
v_05_08_24_exp1_B2 6 0.895 0.895 0.895
v_05_08_24_exp1_B2 60 0.924 0.924 0.924
v_05_08_24_exp2_C1 1 0.857 0.857 0.857
v_05_08_24_exp2_C1 6 0.889 0.889 0.889
v_05_08_24_exp2_C1 60 0.925 0.925 0.925
v_05_08_24_exp2_D1 1 0.971 0.971 0.971
v_05_08_24_exp2_D1 6 0.986 0.986 0.986
v_05_08_24_exp2_D1 60 0.998 0.998 0.998
v_05_08_24_exp2_A2 1 0.857 0.857 0.857
v_05_08_24_exp2_A2 6 0.914 0.914 0.914
v_05_08_24_exp2_A2 60 0.961 0.961 0.961
v_05_08_24_exp2_B2 1 0.924 0.924 0.924
v_05_08_24_exp2_B

v_07_24_24_exp2_D1 1 0.819 0.819 0.819
v_07_24_24_exp2_D1 6 0.898 0.898 0.898
v_07_24_24_exp2_D1 60 0.954 0.954 0.954
v_07_24_24_exp2_A2 1 0.988 0.988 0.988
v_07_24_24_exp2_A2 6 0.995 0.995 0.995
v_07_24_24_exp2_A2 60 0.999 0.999 0.999
v_07_24_24_exp2_B2 1 0.951 0.951 0.951
v_07_24_24_exp2_B2 6 0.970 0.970 0.970
v_07_24_24_exp2_B2 60 0.983 0.983 0.983
v_07_24_24_exp2_C2 1 0.815 0.815 0.815
v_07_24_24_exp2_C2 6 0.845 0.845 0.845
v_07_24_24_exp2_C2 60 0.876 0.876 0.876
v_07_24_24_exp2_D2 1 0.903 0.903 0.903
v_07_24_24_exp2_D2 6 0.935 0.935 0.935
v_07_24_24_exp2_D2 60 0.969 0.969 0.969
v_07_24_24_exp3_A1 1 0.965 0.965 0.965
v_07_24_24_exp3_A1 6 0.971 0.971 0.971
v_07_24_24_exp3_A1 60 0.981 0.981 0.981
v_07_24_24_exp3_B1 1 0.967 0.967 0.967
v_07_24_24_exp3_B1 6 0.989 0.989 0.989
v_07_24_24_exp3_B1 60 0.997 0.997 0.997
v_07_24_24_exp3_C1 1 0.766 0.766 0.766
v_07_24_24_exp3_C1 6 0.784 0.784 0.784
v_07_24_24_exp3_C1 60 0.812 0.812 0.812
v_07_24_24_exp3_D1 1 0.706 0.706 0.706
v_07_24_24_exp3_D

In [5]:
np.set_printoptions(precision=3)

window_size = [1,6,60]
# window_size = [30,60]

l_ws = len(window_size)

filter_status = '_filtered'
# filter_status = ''

thr_presence = 0.1
size_npz = shared_data_folder+exp_target+'_bodysize_event.npz'
loaded_size = np.load(size_npz)
well_done_rate_f_2bp_selected, well_done_rate_w_2bp_selected, well_done_rate_c_selected, well_done_rate_temp_2bp_selected, well_done_rate_temp_selected = (mcam_tools.all_tracking_data_rate(
    h5_dir, loaded_size, filter_status, scorer_floor, scorer_wall, scorer_cross,experiments_mscore, experiments_index, fileloc_set, videofile_set, 
    animalID, bodyparts_floor, bodyparts_wall, bodyparts_cross, num_file,num_well, num_exp, window_size, crop_h, frame_n, frame_n_e, thr_presence))



v_05_08_24_exp1_A1 1020 547
v_05_08_24_exp1_A1 1 108000.0 0.212 0.245 0.219 0.498 0.879
v_05_08_24_exp1_A1 6 18000.0 0.213 0.298 0.230 0.543 0.914
v_05_08_24_exp1_A1 60 1800.0 0.218 0.407 0.280 0.625 0.951
v_05_08_24_exp1_B1 167 747
v_05_08_24_exp1_B1 1 108000.0 0.211 0.497 0.050 0.749 0.920
v_05_08_24_exp1_B1 6 18000.0 0.219 0.589 0.060 0.796 0.950
v_05_08_24_exp1_B1 60 1800.0 0.251 0.734 0.099 0.876 0.974
v_05_08_24_exp1_D1 389 1533
v_05_08_24_exp1_D1 1 108000.0 0.135 0.394 0.024 0.572 0.731
v_05_08_24_exp1_D1 6 18000.0 0.147 0.486 0.037 0.660 0.836
v_05_08_24_exp1_D1 60 1800.0 0.186 0.709 0.093 0.832 0.943
v_05_08_24_exp1_A2 1036 397
v_05_08_24_exp1_A2 1 108000.0 0.094 0.411 0.035 0.661 0.950
v_05_08_24_exp1_A2 6 18000.0 0.102 0.515 0.045 0.729 0.976
v_05_08_24_exp1_A2 60 1800.0 0.128 0.666 0.085 0.818 0.994
v_05_08_24_exp1_B2 18 741
v_05_08_24_exp1_B2 1 108000.0 0.000 0.414 0.289 0.433 0.865
v_05_08_24_exp1_B2 6 18000.0 0.000 0.471 0.300 0.491 0.895
v_05_08_24_exp1_B2 60 1800.0 0.0

v_05_15_24_exp1_C2 6 18000.0 0.247 0.449 0.141 0.804 0.951
v_05_15_24_exp1_C2 60 1800.0 0.254 0.531 0.156 0.850 0.970
v_05_15_24_exp1_D2 0 3586
v_05_15_24_exp1_D2 1 108000.0 0.309 0.160 0.036 0.781 0.958
v_05_15_24_exp1_D2 6 18000.0 0.315 0.215 0.038 0.818 0.970
v_05_15_24_exp1_D2 60 1800.0 0.334 0.306 0.045 0.869 0.982
v_05_17_24_exp1_A1 82 740
v_05_17_24_exp1_A1 1 108000.0 0.504 0.201 0.005 0.906 0.966
v_05_17_24_exp1_A1 6 18000.0 0.512 0.291 0.007 0.927 0.978
v_05_17_24_exp1_A1 60 1800.0 0.551 0.436 0.012 0.960 0.991
v_05_17_24_exp1_B1 177 5378
v_05_17_24_exp1_B1 1 108000.0 0.187 0.244 0.020 0.596 0.731
v_05_17_24_exp1_B1 6 18000.0 0.201 0.304 0.025 0.645 0.775
v_05_17_24_exp1_B1 60 1800.0 0.250 0.488 0.052 0.764 0.842
v_05_17_24_exp1_C1 2354 277
v_05_17_24_exp1_C1 1 108000.0 0.200 0.585 0.017 0.836 0.943
v_05_17_24_exp1_C1 6 18000.0 0.210 0.669 0.026 0.875 0.967
v_05_17_24_exp1_C1 60 1800.0 0.243 0.828 0.057 0.957 0.989
v_05_17_24_exp1_D1 895 992
v_05_17_24_exp1_D1 1 108000.0 0.231

v_07_24_24_exp3_B2 6 18000.0 0.087 0.666 0.037 0.778 0.924
v_07_24_24_exp3_B2 60 1800.0 0.110 0.793 0.072 0.872 0.968
v_07_24_24_exp3_C2 1965 1348
v_07_24_24_exp3_C2 1 108000.0 0.201 0.537 0.046 0.655 0.910
v_07_24_24_exp3_C2 6 18000.0 0.215 0.599 0.050 0.694 0.936
v_07_24_24_exp3_C2 60 1800.0 0.259 0.700 0.060 0.760 0.954
v_07_24_24_exp4_A1 97 377
v_07_24_24_exp4_A1 1 108000.0 0.001 0.561 0.118 0.682 0.919
v_07_24_24_exp4_A1 6 18000.0 0.002 0.620 0.131 0.736 0.963
v_07_24_24_exp4_A1 60 1800.0 0.008 0.697 0.148 0.803 0.986
v_07_24_24_exp4_A2 734 1410
v_07_24_24_exp4_A2 1 108000.0 0.094 0.453 0.078 0.594 0.819
v_07_24_24_exp4_A2 6 18000.0 0.097 0.524 0.096 0.653 0.864
v_07_24_24_exp4_A2 60 1800.0 0.106 0.656 0.142 0.763 0.915
v_07_24_24_exp4_B2 3617 326
v_07_24_24_exp4_B2 1 108000.0 0.016 0.384 0.230 0.454 0.870
v_07_24_24_exp4_B2 6 18000.0 0.021 0.503 0.270 0.586 0.913
v_07_24_24_exp4_B2 60 1800.0 0.034 0.653 0.332 0.732 0.947
v_07_24_24_exp4_C2 15 634
v_07_24_24_exp4_C2 1 108000.0 0.4

In [6]:
print(np.mean(t_face_tracked, axis=1)/1800, np.mean(t_chase_tracked, axis=1)/1800)

[0.893 0.927 0.959] [0.893 0.927 0.959]


In [7]:
# d_f1f2_m_tracked,
# t_f1f2_m_tracked,

# ad_d_f1f2_m_tracked,
# t_ad_d_f1f2_m_tracked,

# t_f1f2_c_tracked,

# d_f1f2_tracked,
# t_f1f2_tracked,

# d_tracked,
# t_speed_tracked,

# norm_f1_f2_face_m, norm_f1_f2_chase_m, t_face_tracked, t_chase_tracked,
# norm_f1_f2_face_m_wocr,norm_f1_f2_chase_m_wocr, t_face_wocr_tracked, t_chase_wocr_tracked


np.savez(shared_data_folder+exp_target+'_locomotion.npz',
         
        d_f1f2_m_tracked = d_f1f2_m_tracked, 
        t_f1f2_m_tracked = t_f1f2_m_tracked,
         
        ad_d_f1f2_m_tracked   = ad_d_f1f2_m_tracked, 
        t_ad_d_f1f2_m_tracked = t_ad_d_f1f2_m_tracked, 
         
        t_f1f2_c_tracked = t_f1f2_c_tracked,
         
        d_f1f2_tracked = d_f1f2_tracked,
        t_f1f2_tracked = t_f1f2_tracked,
        

        d_tracked = d_tracked,
        t_speed_tracked = t_speed_tracked,
        
        norm_f1_f2_chase_m = norm_f1_f2_chase_m,
        norm_f1_f2_face_m  = norm_f1_f2_face_m,
        t_chase_tracked    = t_chase_tracked,
        t_face_tracked     = t_face_tracked,
        norm_f1_f2_chase_m_wocr = norm_f1_f2_chase_m_wocr,
        norm_f1_f2_face_m_wocr  = norm_f1_f2_face_m_wocr,
        t_chase_wocr_tracked    = t_chase_wocr_tracked,
        t_face_wocr_tracked     = t_face_wocr_tracked, 

        total_events = total_events,
        genders_status = genders_status,
        genotypes_status = genotypes_status,
        radius = radius,
        )

In [8]:
np.savez(shared_data_folder+exp_target+'_tracking_rate.npz', 
         well_done_rate_f_2bp_selected = well_done_rate_f_2bp_selected, 
         well_done_rate_w_2bp_selected = well_done_rate_w_2bp_selected, 
         well_done_rate_c_selected = well_done_rate_c_selected, 
         well_done_rate_temp_2bp_selected = well_done_rate_temp_2bp_selected, 
         well_done_rate_temp_selected = well_done_rate_temp_selected
        )

In [9]:
np.set_printoptions(precision=3)
print(np.mean(well_done_rate_f_2bp_selected   ,axis=1))
print(np.mean(well_done_rate_w_2bp_selected   ,axis=1))
print(np.mean(well_done_rate_c_selected       ,axis=1))
print(np.mean(well_done_rate_temp_2bp_selected,axis=1))
print(np.mean(well_done_rate_temp_selected    ,axis=1))

[0.249 0.258 0.286]
[0.406 0.486 0.621]
[0.039 0.046 0.071]
[0.743 0.79  0.862]
[0.892 0.927 0.959]


In [8]:
t_f1f2_c_tracked_total = np.sum(t_f1f2_c_tracked,axis=2)

for i_sex in range(2):
    gender = (genders_status  ==i_sex)
    csv_block = -np.ones((np.sum(gender)+3,10*l_ws))
    csv_block[:,:] = np.NaN
    i_start = 0
    for j_gtype in range(3):
        gtype = (genotypes_status==j_gtype)
        g_g   = np.logical_and(gender, gtype)
        n_gt  = np.sum(g_g)
        j_start = 0
        for i in range(l_ws):

            
            speed = d_tracked[i,g_g]/t_speed_tracked[i,g_g]
            
#             print(i_sex,j_gtype,i, csv_block[i_start:i_start+n_gt,j_start+0].shape, speed.shape)
            
            csv_block[i_start:i_start+n_gt,j_start+0] = speed

            d_center = d_f1f2_m_tracked[i,g_g]
            csv_block[i_start:i_start+n_gt,j_start+1] = d_center

            t = t_f1f2_m_tracked[i,g_g]
            csv_block[i_start:i_start+n_gt,j_start+2] = t

            d_d_hh_tt = d_d_f1f2_hh_tt_m_tracked[i,g_g]
            csv_block[i_start:i_start+n_gt,j_start+3] = d_d_hh_tt

#             t = t_tracked_ht[i,g_g]
#             csv_block[i_start:i_start+n_gt,j_start+4] = t

            ratio = t_f1f2_c_tracked[i,g_g,1]/t_f1f2_c_tracked_total[i,g_g]
            csv_block[i_start:i_start+n_gt,j_start+4] = ratio
            
            

            
#             d_f_ht = d_f_ht_tracked[i,g_g,:]
#             csv_block[i_start:i_start+n_gt,(j_start+6):(j_start+8)] = d_f_ht
#             print(i_start, j_start)
            
            j_start += 10

#         csv_block[i_start:i_start+n_gt,5:8] = toto_event_g_g
        i_start += (sum(g_g)+1)
    np.savetxt(shared_data_folder+exp_target+'_'+sex_name[i_sex]+'_locomotion.csv', csv_block, delimiter=',', header=exp_target+'_'+sex_name[i_sex])


In [4]:
np.set_printoptions(precision=2)

window_size = [60]

l_ws = len(window_size)

filter_status = '_filtered'


thr_presence = 0.1
size_npz = shared_data_folder+exp_target+'_bodysize_event.npz'
loaded_size = np.load(size_npz)
d_f1f2_m_tracked, t_f1f2_m_tracked, ad_d_f1f2_m_tracked, t_ad_d_f1f2_m_tracked, t_f1f2_c_tracked, d_f1f2_tracked, t_f1f2_tracked, d_tracked, t_speed_tracked,norm_f1_f2_face_m, norm_f1_f2_chase_m, t_face_tracked, t_chase_tracked,norm_f1_f2_face_m_wocr,norm_f1_f2_chase_m_wocr, t_face_wocr_tracked, t_chase_wocr_tracked = (mcam_tools.all_tracking_data(
    h5_dir, loaded_size, filter_status, scorer_floor, scorer_wall, scorer_cross,experiments_mscore, experiments_index, fileloc_set, videofile_set, 
    animalID, bodyparts_floor, num_file,num_well, num_exp, window_size, crop_h, frame_n, frame_n_e, thr_presence))


v_05_08_24_exp1_A1 60 0.891 0.891 0.891
v_05_08_24_exp1_B1 60 0.976 0.976 0.976
v_05_08_24_exp1_D1 60 0.932 0.932 0.932
v_05_08_24_exp1_A2 60 0.994 0.994 0.994
v_05_08_24_exp1_B2 60 0.923 0.923 0.923
v_05_08_24_exp2_C1 60 0.918 0.918 0.918
v_05_08_24_exp2_D1 60 0.991 0.991 0.991
v_05_08_24_exp2_A2 60 0.961 0.961 0.961
v_05_08_24_exp2_B2 60 0.986 0.986 0.986


KeyboardInterrupt: 